<a href="https://colab.research.google.com/github/garvjain7/Chatbot/blob/main/Match_winner_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('IPL Matches 2008-2020.csv')

# Display basic information about the dataset
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               816 non-null    int64  
 1   city             803 non-null    object 
 2   date             816 non-null    object 
 3   player_of_match  812 non-null    object 
 4   venue            816 non-null    object 
 5   neutral_venue    816 non-null    int64  
 6   team1            816 non-null    object 
 7   team2            816 non-null    object 
 8   toss_winner      816 non-null    object 
 9   toss_decision    816 non-null    object 
 10  winner           812 non-null    object 
 11  result           812 non-null    object 
 12  result_margin    799 non-null    float64
 13  eliminator       812 non-null    object 
 14  method           19 non-null     object 
 15  umpire1          816 non-null    object 
 16  umpire2          816 non-null    object 
dtypes: float64(1), i

(None,
        id        city        date player_of_match  \
 0  335982   Bangalore  2008-04-18     BB McCullum   
 1  335983  Chandigarh  2008-04-19      MEK Hussey   
 2  335984       Delhi  2008-04-19     MF Maharoof   
 3  335985      Mumbai  2008-04-20      MV Boucher   
 4  335986     Kolkata  2008-04-20       DJ Hussey   
 
                                         venue  neutral_venue  \
 0                       M Chinnaswamy Stadium              0   
 1  Punjab Cricket Association Stadium, Mohali              0   
 2                            Feroz Shah Kotla              0   
 3                            Wankhede Stadium              0   
 4                                Eden Gardens              0   
 
                          team1                        team2  \
 0  Royal Challengers Bangalore        Kolkata Knight Riders   
 1              Kings XI Punjab          Chennai Super Kings   
 2             Delhi Daredevils             Rajasthan Royals   
 3               Mu

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [1]:
# Load dataset
df = pd.read_csv('IPL Matches 2008-2020.csv')

# Data Cleaning: Handling Missing Values
df['city'].fillna('Unknown', inplace=True)
df.dropna(subset=['winner'], inplace=True)
df.fillna({'result_margin': df['result_margin'].median(), 'player_of_match': 'Unknown', 'method': 'Unknown'}, inplace=True)

# Remove Outliers in result_margin (wins by huge margins)
q_low = df['result_margin'].quantile(0.01)
q_high = df['result_margin'].quantile(0.99)
df = df[(df['result_margin'] >= q_low) & (df['result_margin'] <= q_high)]

# Feature Engineering: Extracting More Information
df['year'] = pd.to_datetime(df['date']).dt.year
df['month'] = pd.to_datetime(df['date']).dt.month
df['season'] = df['year'].apply(lambda x: f"IPL {x}")

# Feature Selection: Choosing relevant columns
features = ['team1', 'team2', 'toss_winner', 'toss_decision', 'venue', 'month']
label = 'winner'

# Encoding Categorical Variables with One-Hot Encoding
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # Ensure output is dense
encoded_array = encoder.fit_transform(df[features])
df_encoded = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(features))
df_encoded[label] = LabelEncoder().fit_transform(df[label])
df_encoded.reset_index(drop=True, inplace=True)

# Splitting Data: Training (2008-2019) and Testing (2020)
train_data = df[df['year'] < 2020]
test_data = df[df['year'] == 2020]

X_train = df_encoded.iloc[train_data.index, :-1]
y_train = df_encoded.iloc[train_data.index, -1]
X_test = df_encoded.iloc[test_data.index, :-1]
y_test = df_encoded.iloc[test_data.index, -1]

# Model Training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Feature Importance
plt.figure(figsize=(10, 5))
sns.barplot(x=model.feature_importances_, y=X_train.columns)
plt.title("Feature Importance in Match Prediction")
plt.show()

# Predicting a future match
def predict_winner(team1, team2, toss_winner, toss_decision, venue, month):
    input_data = pd.DataFrame([[team1, team2, toss_winner, toss_decision, venue, month]], columns=features)
    input_data = pd.DataFrame(encoder.transform(input_data), columns=encoder.get_feature_names_out(features))
    winner = model.predict(input_data)[0]
    return LabelEncoder().inverse_transform([winner])[0]

# Example Prediction
predicted_winner = predict_winner('Mumbai Indians', 'Chennai Super Kings', 'Mumbai Indians', 'bat', 'Wankhede Stadium', 4)
print(f"Predicted Winner: {predicted_winner}")


NameError: name 'pd' is not defined

In [10]:
print(set(train_data.index) - set(df_encoded.index))  # Should be empty
print(set(test_data.index) - set(df_encoded.index))   # Should be empty

set()
{803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815}
